### 数据源：https://joras.csis.u-tokyo.ac.jp/dataset/show/id/14000202000
### 目标：提取首都圈的大型店铺/娱乐场所/餐厅的POI 信息

In [1]:
import pandas as pd

#### Step 1. 数据读取 
- 仅保留 店名，类型码，经度，纬度

In [2]:
import pandas as pd
import os
path_1 = "/path/11"
path_2 = "/path/12"
path_3 = "/path/13"
path_4 = "/path/14"
all_files_1 = [os.path.join(path_1, x) for x in os.listdir(path_1)]
all_files_2 = [os.path.join(path_2, x) for x in os.listdir(path_2)]
all_files_3 = [os.path.join(path_3, x) for x in os.listdir(path_3)]
all_files_4 = [os.path.join(path_4, x) for x in os.listdir(path_4)]
all_files = all_files_1 + all_files_2 + all_files_3 + all_files_4
li = []
for filename in all_files:   # 注意 类别码 是字符串格式
    df = pd.read_csv(filename, index_col=None, header = None,encoding= 'shift-JIS',usecols=[0,10,20,21],dtype={10:'str'})
    li.append(df)
data = pd.concat(li, axis=0, ignore_index=True)

In [3]:
data.columns = ['name','category','lon','lat']

#### Step 2. 过滤掉 个人电话用户 

In [5]:
data = data[data['category'].notnull()]

In [6]:
data

,name,category,lon,lat
1,（株）アイビックス 大宮工場,1601000,139.589022,35.894541
2,日本郵便輸送（株）さいたま物流センター,2509000,139.586545,35.890433
3,さいたまメモリアルパーク,3571000,139.584468,35.895304
23,中村金物店,1302000,139.593366,35.893898
25,植水郵便局,3821000,139.594448,35.894767
...,...,...,...,...
4303503,宮ヶ瀬水の郷交流館,3818000,139.228760,35.523315
4303504,（有）フリート,3173000,139.227249,35.520674
4303505,（財）宮ヶ瀬ダム周辺振興財団,3902000,139.227624,35.522019
4303507,宮ヶ瀬湖遊覧船案内所,2303000,139.227624,35.522019


#### Step 3. 去重

In [8]:
data = data.drop_duplicates()
data

,name,category,lon,lat
1,（株）アイビックス 大宮工場,1601000,139.589022,35.894541
2,日本郵便輸送（株）さいたま物流センター,2509000,139.586545,35.890433
3,さいたまメモリアルパーク,3571000,139.584468,35.895304
23,中村金物店,1302000,139.593366,35.893898
25,植水郵便局,3821000,139.594448,35.894767
...,...,...,...,...
4303503,宮ヶ瀬水の郷交流館,3818000,139.228760,35.523315
4303504,（有）フリート,3173000,139.227249,35.520674
4303505,（財）宮ヶ瀬ダム周辺振興財団,3902000,139.227624,35.522019
4303507,宮ヶ瀬湖遊覧船案内所,2303000,139.227624,35.522019


#### Step 4. 坐标偏移矫正，从日本坐标系 矫正到 世界坐标系

In [9]:
def get_global_lat(jlat,jlon):
    glat = (jlat - jlat * 0.00010695 + jlon * 0.000017464 + 0.0046017)
    return glat
def get_global_lon(jlat,jlon):
    glon = jlon - jlat * 0.000046038 - jlon * 0.000083043 + 0.010040
    return glon
def convert_coords(df):
    df['n_lat'] = df.apply(lambda x: get_global_lat(x['lat'],x['lon']), axis=1)
    df['n_lon'] = df.apply(lambda x: get_global_lon(x['lat'],x['lon']), axis=1)
    return df

data = convert_coords(data)

/home/chuang/anaconda3/envs/vis20/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/chuang/anaconda3/envs/vis20/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [10]:
data = data[['name','category','n_lat','n_lon']]
data.columns = ['name','category','lat','lon']
data

,name,category,lat,lon
1,（株）アイビックス 大宮工場,1601000,35.897742,139.585818
2,日本郵便輸送（株）さいたま物流センター,2509000,35.893634,139.583341
3,さいたまメモリアルパーク,3571000,35.898505,139.581264
23,中村金物店,1302000,35.897099,139.590162
25,植水郵便局,3821000,35.897968,139.591243
...,...,...,...,...
4303503,宮ヶ瀬水の郷交流館,3818000,35.526549,139.225602
4303504,（有）フリート,3173000,35.523908,139.224092
4303505,（財）宮ヶ瀬ダム周辺振興財団,3902000,35.525253,139.224467
4303507,宮ヶ瀬湖遊覧船案内所,2303000,35.525253,139.224467


#### Step 5. 提取 娱乐和食事关联 还有 大型综合店铺 两大类
![jupyter](./categories.png)
- 并保存到./entertainment_restaurant_shoppingmall_poi.csv 

In [11]:
def fxck(type_code):
    if type_code.startswith("31"):
        return True
    if type_code.startswith("34"):
        return True
    return False
data['newcolumn'] = data.apply(lambda x: fxck(x['category']), axis=1)

/home/chuang/anaconda3/envs/vis20/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [13]:
filtered_poi = data.loc[data['newcolumn'] == True]
del filtered_poi['newcolumn']
filtered_poi

,name,category,lat,lon
142,Ｐ．Ｓ．Ｘさいたま西区店,3106000,35.885870,139.556161
294,船渡,3166000,35.883116,139.554523
431,和風スナック花,3173000,35.888989,139.596207
474,池田屋,3132000,35.888167,139.596186
655,銀鮨,3149000,35.890896,139.588314
...,...,...,...,...
4303455,民芸風食事処・恵山亭,3173000,35.524363,139.223677
4303461,山登,3132000,35.524184,139.223744
4303478,びんずる,3167000,35.523366,139.224143
4303491,田舎料理水の郷,3173000,35.526549,139.225602


In [14]:
filtered_poi.to_csv("./entertainment_restaurant_shoppingmall_poi_with_city_code.csv", encoding='utf-8', index=False)

#### Step 6. 计算每个网格中总POI 的数目

In [15]:
import h3 
all_poi = data[['name','category','lat','lon']]
all_poi['hid']  = all_poi.apply(lambda x: h3.geo_to_h3(x['lat'],x['lon'],8), axis=1)

In [16]:
all_poi

,name,category,lat,lon,hid
1,（株）アイビックス 大宮工場,1601000,35.897742,139.585818,882f5a2e59fffff
2,日本郵便輸送（株）さいたま物流センター,2509000,35.893634,139.583341,882f5a2e59fffff
3,さいたまメモリアルパーク,3571000,35.898505,139.581264,882f5a2e59fffff
23,中村金物店,1302000,35.897099,139.590162,882f5a20b7fffff
25,植水郵便局,3821000,35.897968,139.591243,882f5a20b7fffff
...,...,...,...,...,...
4303503,宮ヶ瀬水の郷交流館,3818000,35.526549,139.225602,882f5bc5d7fffff
4303504,（有）フリート,3173000,35.523908,139.224092,882f5bc5d3fffff
4303505,（財）宮ヶ瀬ダム周辺振興財団,3902000,35.525253,139.224467,882f5bc5d3fffff
4303507,宮ヶ瀬湖遊覧船案内所,2303000,35.525253,139.224467,882f5bc5d3fffff


In [17]:
res = all_poi.groupby(['hid']).size().reset_index(name='counts')
res

,hid,counts
0,882e2da413fffff,81
1,882e2da417fffff,6
2,882e2da41bfffff,14
3,882e2da481fffff,1
4,882e2da483fffff,2
...,...,...
15034,882f5e2eb1fffff,1
15035,882f5e2eb3fffff,31
15036,882f5e2eb5fffff,2
15037,882f5e2eb7fffff,9


In [18]:
res.to_csv("./poi_count_by_hid.csv", encoding='utf-8', index=False)